# Лабораторная работа № 2

# Блочное симметричное шифрование

## Вариант

3(в)

## Задание 

Реализовать систему симметричного блочного шифрования,
позволяющую шифровать и дешифровать файл на диске с использованием
блочного шифра RC6 в режиме шифрования PCBC. 

# Константы

$ w $ - размер слова в битах. Блоки открытого текста и шифротекста имеют длину $ 4w $ - бит (т.к. 4 регистра).

$ r $ - количество раундов. Так же размер расширенного массива ключей $ S $ будет иметь $ t=(2r+4) $ слово. Допустимые значения $ 0\leq r\leq 255 $. 

$ b $ - количество бит в секретном ключе $ K $.

$ K $ - секретный ключ размеров $ b $ бит: $K[0], K[1], ... , K[b-1] $.

In [59]:
import math

w = 32
r = 2
b = 8
K = [0, 1, 0, 1, 0, 1, 0, 1]

# количество байт в слове
u = w/8

# Подготовка ключа

генерация констант

Для конкретного w мы определяем две величины:

$$ P_w \leftarrow Odd((e-2)2^w)\\Q_w \leftarrow Odd((\phi-1)2^w) $$

где $ e=2.71828... $(экспонента), $ \phi=1.61803... $ (золотое сечение). 

$ Odd(\cdot) $ - операция округления до ближайшего нечетного целого.

In [28]:
# операция округления до ближайшего нечетного целого
def odd(x):
    if x - math.trunc(x) == 0:
        return x + 1
    return math.ceil(x) if math.ceil(x) % 2 == 1 else math.floor(x)

# константа золотого сечения
phi = (1 + 5 ** 0.5) / 2

# генерация констант
p_w = bin( odd( (math.e - 2) * math.pow(2, w) ) )
q_w = bin( odd( (phi - 1) * math.pow(2, w) ) )

print(f'P_w = {p_w}\nQ_w = {q_w}')

P_w = 0b10110111111000010101000101100011
Q_w = 0b10011110001101110111100110111001


# Конвертация секретного ключа

На первом этапе нужно скопировать секретный ключ из массива $ K[0...b-1] $ в массив $ L[0...c-1] $, который состоит из $c=b/u$ слов, где $u=w/8$-количество байт в слове. Если   $b$ не кратен $w / 8$, то $L$дополняется нулевыми битами до ближайшего большего кратного:

In [63]:
c = math.ceil(max(b, 1) / u)

L = [[]]
for i in range(0, c):
    L[i] = []
    for j in range(0, math.trunc(b/c)):
        L[i].append(K[math.trunc(i * u + j)] if i * u + j < b else 0)
    if i == c - 1:
        break
    L.append([])
print(f'Конвертация секретного ключа: {L}')

Конвертация секретного ключа: [[0, 1, 0, 1], [0, 1, 0, 1]]
